In [1]:
import numpy as np
import pandas as pd
import plotly

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3

conn = sqlite3.connect("stock.sqlite")
cur = conn.cursor()
ticker1 = 'AOT.BK'
ticker2 = 'ACE.BK'
ticker3 = 'KEX.BK'
ticker4 = 'EPG.BK'
ticker5 = 'AAV.BK'
ticker6 = 'DELTA.BK'
query1 = "select * from stock_table_hr where `ticker` == '%s'" % ticker1
query2 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-04' and datetime < '2023-01-06'" % ticker2
query3 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-05' and datetime < '2023-01-06'" % ticker3
query4 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-05' and datetime < '2023-01-06'" % ticker4
query5 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-05' and datetime < '2023-01-06'" % ticker5
query6 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2022-12-20' and datetime < '2023-01-06'" % ticker6
AOT = pd.read_sql(query1,conn)
ACE = pd.read_sql(query2,conn)
KEX = pd.read_sql(query3,conn)
EPG = pd.read_sql(query4,conn)
AAV = pd.read_sql(query5,conn)
DELTA = pd.read_sql(query6,conn)
x = [ticker1,ticker2,ticker3,ticker4,ticker5,ticker6]
count = 0
#declare figure
fig = go.Figure()
DELTA['time'] = pd.to_datetime(DELTA['Datetime'])
DELTA.set_index(DELTA['time'],inplace = True)

#Candlestick
for column in [AOT,ACE,KEX,EPG,AAV,DELTA]:
    # name = column.Ticker[0]
    fig.add_trace(go.Candlestick(x=column.Datetime,
                    open=column.Open,
                    high=column.High,
                    low=column.Low,
                    close=column.Close, 
                    name = x[count],))
    count += 1
    
df_resample = DELTA.resample('H').max()
merged_index  = DELTA.index.append(df_resample.index)
timegap = merged_index[~merged_index.duplicated(keep = False)]
fig.update_xaxes( rangebreaks=[ dict(values = timegap, dvalue = 3600000) ])

# Add titles
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'Disable',
                method = 'update',
                args = [{'visible': [False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                        {'title': 'Disable',
                        'showlegend':True}]),

            dict(label = 'AOT',
                method = 'update',
                args = [{'visible': [True, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                        {'title': 'AOT',
                        'showlegend':True}]),
            dict(label = 'ACE',
                method = 'update',
                args = [{'visible': [False, True, False, False, False, False]},
                        {'title': 'ACE',
                        'showlegend':True}]),
            dict(label = 'KEX',
                method = 'update',
                args = [{'visible': [False, False, True, False, False, False]},
                        {'title': 'KEX',
                        'showlegend':True}]),
            dict(label = 'EPG',
                method = 'update',
                args = [{'visible': [False, False, False, True, False, False]},
                        {'title': 'EPG',
                        'showlegend':True}]),
            dict(label = 'AAV',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, False]},
                          {'title': 'AAV',
                           'showlegend':True}]),
            dict(label = 'DELTA',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, True]},
                          {'title': 'DELTA',
                           'showlegend':True}]),
            ])
        )
    ])

fig.update_traces(visible="legendonly")
fig.update_yaxes(fixedrange=False)
# X-Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15m", step="minute", stepmode="backward"),
            dict(count=45, label="45m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="3h", step="hour", stepmode="backward"),
            dict(step="all")
        ])
    )
)

#Show
fig.show()

In [10]:
DELTA['time'] = pd.to_datetime(DELTA['Datetime'])
DELTA.set_index(DELTA['time'],inplace = True)
df_resample = DELTA.resample('H').max()
merged_index  = DELTA.index.append(df_resample.index)
timegap = merged_index[~merged_index.duplicated(keep = False)]
print(timegap)

DatetimeIndex(['2022-12-01 01:00:00', '2022-12-01 02:00:00',
               '2022-12-01 03:00:00', '2022-12-01 04:00:00',
               '2022-12-01 05:00:00', '2022-12-01 06:00:00',
               '2022-12-01 07:00:00', '2022-12-01 08:00:00',
               '2022-12-01 09:00:00', '2022-12-01 10:00:00',
               ...
               '2023-01-04 14:00:00', '2023-01-04 15:00:00',
               '2023-01-04 16:00:00', '2023-01-04 17:00:00',
               '2023-01-04 18:00:00', '2023-01-04 19:00:00',
               '2023-01-04 20:00:00', '2023-01-04 21:00:00',
               '2023-01-04 22:00:00', '2023-01-04 23:00:00'],
              dtype='datetime64[ns]', name='time', length=818, freq=None)


In [ ]:
from datetime import timedelta
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

df = yf.download(tickers='DELTA.BK', period='20d', interval='1h')
close = df['Close']
fig = go.Figure()
fig.add_trace(go.Candlestick(x=df.index,
                open=df.Open,
                high=df.High,
                low=df.Low,
                close=df.Close, name = 'market data'))
df['EMA12'] = df['Close'].ewm(span=12, adjust=False, min_periods = 0).mean()
df['EMA26'] = df['Close'].ewm(span=26, adjust=False, min_periods = 0).mean()
# create a new column 'Signal' such that if 20-day EMA is greater   # than 50-day EMA then set Signal as 1 else 0

df['Signal'] = 0.0  
df['Signal'] = np.where(df['EMA12'] > df['EMA26'], 1.0, 0.0)
# create a new column 'Position' which is a day-to-day difference of # the 'Signal' column
df['Position'] = df['Signal'].diff()

fig.add_trace(go.Scatter(name = 'EMA12',x=df.index,y=df['EMA12']))
fig.add_trace(go.Scatter(name = 'EMA26',x=df.index,y=df['EMA26']))
fig.add_trace(go.Scatter(mode="markers",marker=dict(size=20, color="LightSeaGreen"),name = 'Buy',x=df[df['Position'] == 1].index,y=df['EMA12'][df['Position'] == 1]))
fig.add_trace(go.Scatter(mode="markers",marker=dict(size=20, color="hotpink"),name = 'Sell',x=df[df['Position'] == -1].index,y=df['EMA12'][df['Position'] == -1]))
# fig.scatter(close.index[idx20], df['EMA26'][idx20], color='red')
df_resample = df.resample('H').max()
merged_index  = df.index.append(df_resample.index)
timegap = merged_index[~merged_index.duplicated(keep = False)]

fig.update_xaxes( rangebreaks=[ dict(values = timegap, dvalue = 3600000) ])

fig.update_yaxes(fixedrange=False)
fig.show()
print(df)

In [26]:
import numpy as np
import pandas as pd
import plotly
from plotly.subplots import make_subplots

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3

conn = sqlite3.connect("stock.sqlite")
cur = conn.cursor()
ticker = input("Enter tickers")
query1 = "select * from stock_table_hr where `ticker` == '%s' and datetime > '2023-01-01' and datetime < '2023-01-06'" % ticker
query2 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2022-05-01' and datetime < '2023-01-06'" % ticker
query3 = "select * from stock_table_mo where `ticker` == '%s' and datetime > '2022-01-05' and datetime < '2023-01-06'" % ticker
Hr = pd.read_sql(query1,conn)
Day = pd.read_sql(query2,conn)
Mo = pd.read_sql(query3,conn)
#declare figure
fig = go.Figure()
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.1, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.2, 0.7])
# Day['time'] = pd.to_datetime(Day['Datetime'])
# Day.set_index(Day['time'],inplace = True)

#Candlestick
for column in [Hr,Day,Mo]:
    # name = column.Ticker[0]
    column['time'] = pd.to_datetime(column['Datetime'])
    column['MA50'] = column['Close'].rolling(window = 50 , min_periods = 0).mean()
    column['MA200'] = column['Close'].rolling(window = 200 , min_periods = 0).mean()
    column.set_index(column['time'],inplace = True)
    fig.add_trace(go.Candlestick(x=column.Datetime,
                    open=column.Open,
                    high=column.High,
                    low=column.Low,
                    close=column.Close, 
                    name = ticker,),row=1, col=1)
    fig.add_trace(go.Scatter(name = 'MA50',x=column.Datetime,y=column['MA50']),row=1, col=1)
    fig.add_trace(go.Scatter(name = 'MA200',x=column.Datetime,y=column['MA200']),row=1, col=1)
    fig.add_trace(go.Bar(x=column.Datetime, y=column.Volume,showlegend=False), row=2, col=1)
    
df_resample_hr = Hr.resample('H').max()
merged_index_hr  = Hr.index.append(df_resample_hr.index)
timegap_hr = merged_index_hr[~merged_index_hr.duplicated(keep = False)]

df_resample_day = Day.resample('D').max()
merged_index_day  = Day.index.append(df_resample_day.index)
timegap_day = merged_index_day[~merged_index_day.duplicated(keep = False)]

# fig.update_xaxes( rangebreaks=[ dict(values = timegap_hr , dvalue = 3600000)])

# fig.update_xaxes(rangebreaks=[dict(values=timegap_day),
#                               dict(values=timegap_hr, dvalue=3600000)])

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'Disable',
                method = 'update',
                args = [{'visible': [False, False, False]}, # the index of True aligns with the indices of plot traces
                        {'title': 'Disable',
                        'showlegend':True}]),
            dict(label = 'Hour',
                method = 'update',
                args = [{'visible': [True, True, True, True, False, False, False, False, False, False, False, False]},
                        {'title': '%s Hour' % ticker,
                        'showlegend':True}]),
            dict(label = 'Day',
                method = 'update',
                args = [{'visible': [False, False, False, False, True, True, True, True, False, False, False, False]},
                        {'title': '%s Day' % ticker,
                        'showlegend':True}]),
            dict(label = 'Mount',
                method = 'update',
                args = [{'visible': [False, False, False, False, False, False, False, False, True, True, True, True]},
                        {'title': '%s Mount' % ticker,
                        'showlegend':True}]),
            ])
        )
    ])
# fig.update_yaxes(rangemode="nonnegative")
# fig.update_xaxes(rangemode="nonnegative")
fig.update_traces(visible="legendonly")
fig.update_yaxes(fixedrange=False)
fig.update_layout(hovermode = "x")
fig.update(layout_xaxis_rangeslider_visible=False)
# fig.update_xaxes(
#         rangeslider_visible=True,
#         rangebreaks=[
#             # NOTE: Below values are bound (not single values), ie. hide x to y
#             dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
#             dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
#             # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
#         ]
#     )

# X-Axes
# fig.update_xaxes(
#     rangeslider_visible=True,
#     rangeselector=dict(
#         buttons=list([
#             dict(count=15, label="15m", step="minute", stepmode="backward"),
#             dict(count=45, label="45m", step="minute", stepmode="backward"),
#             dict(count=1, label="HTD", step="hour", stepmode="todate"),
#             dict(count=3, label="3h", step="hour", stepmode="backward"),
#             dict(step="all")
#         ])
#     )
# )

#Show
fig.show()

In [ ]:
import numpy as np
import pandas as pd
import plotly
from plotly.subplots import make_subplots

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3

query1 = 'select Ticker from stock_info'
conn = sqlite3.connect("stock.sqlite")
cur = conn.cursor()
ticker_list = pd.read_sql(query1,conn)
ticker_list = ticker_list['Ticker'].values.tolist()
fig = go.Figure()
SUM_NET = []
for i in ticker_list:
    query2 = "select * from stock_financial where `Ticker` == '%s'" % i
    data = pd.read_sql(query2,conn)
    SNP = data['Net Profit'].values.tolist()
    SNP = (np.nansum(SNP))
    SUM_NET.append(SNP)
# print(SUM_NET)
data = pd.DataFrame({
    'Ticker' : ticker_list,
    'S_NetProfit' : SUM_NET
})
data["Color"] = np.where(data["S_NetProfit"]<0, 'red', 'green')
fig.add_trace(go.Bar(x=data["Ticker"],y=data["S_NetProfit"],marker_color=data["Color"],name='Net Profit'))
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})

In [60]:
from datetime import timedelta
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import datetime

df = yf.download(tickers='AOT.BK', period='50d', interval='1h')
close = df['Close']
fig = go.Figure()
fig.add_trace(go.Candlestick(x=df.index,
                open=df.Open,
                high=df.High,
                low=df.Low,
                close=df.Close, name = 'market data'))
df['EMA12'] = df['Close'].ewm(span=12, adjust=False, min_periods = 0).mean()
df['EMA26'] = df['Close'].ewm(span=26, adjust=False, min_periods = 0).mean()
# create a new column 'Signal' such that if 20-day EMA is greater   # than 50-day EMA then set Signal as 1 else 0

df['Signal'] = 0.0  
df['Signal'] = np.where(df['EMA12'] > df['EMA26'], 1.0, 0.0)
# create a new column 'Position' which is a day-to-day difference of # the 'Signal' column
df['Position'] = df['Signal'].diff()

fig.add_trace(go.Scatter(name = 'EMA12',x=df.index,y=df['EMA12']))
fig.add_trace(go.Scatter(name = 'EMA26',x=df.index,y=df['EMA26']))
fig.add_trace(go.Scatter(mode="markers",marker=dict(size=20, color="LightSeaGreen"),name = 'Buy',x=df[df['Position'] == 1].index,y=df['EMA12'][df['Position'] == 1]))
fig.add_trace(go.Scatter(mode="markers",marker=dict(size=20, color="hotpink"),name = 'Sell',x=df[df['Position'] == -1].index,y=df['EMA12'][df['Position'] == -1]))
# fig.scatter(close.index[idx20], df['EMA26'][idx20], color='red')
df_resample = df.resample('H').max()
merged_index  = df.index.append(df_resample.index)
timegap = merged_index[~merged_index.duplicated(keep = False)]

fig.update_xaxes( rangebreaks=[ dict(values = timegap, dvalue = 3600000) ])

fig.update_yaxes(fixedrange=False)
fig.show()
print(df[df['Position'] == 1].index[-1])
b = df[df['Position'] == 1].index[-1]
s = df[df['Position'] == -1].index[-1]
n = datetime.datetime.now()
piority = 0
if b > s:
    piority = piority + 1
    print('Golden crossing')
elif b < s:
    piority = piority - 2
    print('Death crossing')
d = datetime.timedelta(days = 20)
p = list(df['Close'][df.index >= (n-d)].values)
for i in range(len(p)-1):
    if p[-1] > p[i]:
        piority = piority + 1
print(piority)
print(abs(piority)/(len(p)+1))

[*********************100%***********************]  1 of 1 completed


2023-02-28 12:00:00
Death crossing
-2
0.02531645569620253
